In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# Introduction:
This notebook is aimed to novices, to show how different sizes of hold-out sets can affect your conclusions and your models.
For this we will use the following Dataframe:

In [2]:
df = pd.read_csv('../input/candy-data.csv')
df.head()

In [3]:
df.describe()

## Objective: see how different sizes of Hold-out sets affect our models.
#### sidenote: since we don't care that much about the quality of the model, little data preprocessing will be done

In [4]:
cols = df.columns[1:]
data = df[cols].values
features = data[:,:-1]
target = data[:,-1]

In [5]:
def normalize(feat):
    return (feat-np.min(feat, axis=0))/(np.max(feat, axis=0)-np.min(feat, axis=0))

In [6]:
n_feat = normalize(features) 

In [7]:
partitions=np.linspace(0.3,0.95,6)
x=np.random.binomial(1,0.8,size=len(df))
train_feat = n_feat[x==1,:]
train_t = target[x==1]
test_feat = n_feat[x==0,:]
test_t = target[x==0]
# Baseline Model
baseline = np.sqrt(np.sum((test_t-np.mean(train_t))**2))/np.sqrt(len(test_t))
print ('The Error of the baseline Model is: {}'.format(baseline))
for prob in partitions:
    print (prob)
    x=np.random.binomial(1,prob,size=len(df))
    train_feat = n_feat[x==1,:]
    train_t = target[x==1]
    test_feat = n_feat[x==0,:]
    test_t = target[x==0]
    lm = LinearRegression()
    lm.fit(train_feat,train_t)
    rmse = np.sqrt(np.sum((test_t-lm.predict(test_feat))**2))/np.sqrt(len(test_t))
    print ('The Error of the linear model is: {}'.format(rmse))
    plt.scatter(test_t,lm.predict(test_feat))
    plt.title('how the predictions are scattered: predicted v/s real target')
    plt.show()


### Conclusions:
If you got too litte data, your model won't be abble to learn the patterns in the data, and this will cause it to perform poorly (In this case, when prob< 0.5 or so). but, since you got a lot of testing data, you will be quite confident that the model is in fact, performing poorly. on te other hand, if you train the model with a  partition of the data too big (prob>0.9) your model might be performing well, but you won't know for sure, because of the high variability of the RMSE in your testing data.
To get a decent estimation of the error just using a hold-out set, you should choose to use about 80% of the data for training, and the rest for testing. Nonetheless, if you few data, perhaps it's better to use more advanced techniques of cross-validation (Leave One Out, etc).
REMEMBER: the production model should be always be trained with the maximum amount of data.